## **Extract features from image using VGG**

### **Import necessary libraries**

In [1]:
import os 
import sys

import pandas as pd 
import numpy as np 

import matplotlib.pyplot as plt 
%matplotlib inline 
import seaborn as sns 

import warnings 
warnings.filterwarnings('ignore')

from PIL import Image 

import torch
import torchvision 
import torchvision.transforms as transforms 
import torchvision.models as models 
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn 
from torch.autograd import Variable


In [2]:
current_cwd = os.getcwd()
lst = current_cwd.split('\\', 2)
new_cwd = '/'.join(lst[:2])
os.chdir(new_cwd)
sys.path.append(new_cwd + '/code')

In [3]:
IMAGES_PATH = 'data/flickr8k/Images'
CAPTIONS_PATH = 'data/flickr8k/captions.txt'

data = pd.read_csv(CAPTIONS_PATH)
data

,image,caption
0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set o...
1,1000268201_693b08cb0e.jpg,A girl going into a wooden building .
2,1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .
3,1000268201_693b08cb0e.jpg,A little girl climbing the stairs to her playh...
4,1000268201_693b08cb0e.jpg,A little girl in a pink dress going into a woo...
...,...,...
40450,997722733_0cb5439472.jpg,A man in a pink shirt climbs a rock face
40451,997722733_0cb5439472.jpg,A man is rock climbing high in the air .
40452,997722733_0cb5439472.jpg,A person in a red shirt climbing up a rock fac...
40453,997722733_0cb5439472.jpg,A rock climber in a red shirt .


In [4]:
images_data = data.drop_duplicates(subset=['image'])
images_data

,image,caption
0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set o...
5,1001773457_577c3a7d70.jpg,A black dog and a spotted dog are fighting
10,1002674143_1b742ab4b8.jpg,A little girl covered in paint sits in front o...
15,1003163366_44323f5815.jpg,A man lays on a bench while his dog sits by him .
20,1007129816_e794419615.jpg,A man in an orange hat starring at something .
...,...,...
40430,990890291_afc72be141.jpg,A man does a wheelie on his bicycle on the sid...
40435,99171998_7cc800ceef.jpg,A group is sitting around a snowy crevasse .
40440,99679241_adc853a5c0.jpg,A grey bird stands majestically on a beach whi...
40445,997338199_7343367d7f.jpg,A person stands near golden walls .


### **Load the VGG11**

In [38]:
# Load the pretrained model
vgg11 = models.vgg11(pretrained=True)

vgg11.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 

In [39]:
# Use the model obeject to select the desired layer
layer = vgg11.classifier[-2]

In [40]:
resize = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])   # mean and std of ImageNet
to_tensor = transforms.ToTensor()

In [41]:
def get_vector_vgg11(img_name):
    # Load the image with Pillow library 
    img = Image.open(img_name).convert('RGB')
    
    # Create a PyTorch Variable with the transformed image 
    transformed_img = Variable(normalize(to_tensor(resize(img))).unsqueeze(0))
    
    # Create a vector of zeros that will hold our feature vector 
    # The 'avgpool' layer has an output size of 4096
    my_embedding = torch.zeros(4096)
    
    # Define a function that will copy the output of a layer 
    def copy_data(m, i, o):
        my_embedding.copy_(o.data.view(-1))
        
    # Attach that function to our selected layer 
    h = layer.register_forward_hook(copy_data)
    
    # Run the model on our transformed image 
    vgg11(transformed_img)
    
    # Detach our copy function from the layer 
    h.remove()
    
    # Return the feature vector 
    return my_embedding

In [42]:
vgg11_image = {}
cnt = 0
for img in images_data.image:
    vgg11_image[img] = get_vector_vgg11(IMAGES_PATH + '/' + img)
    cnt += 1
    if cnt % 1000 == 0:
        print(f"Completed {cnt} images")

Completed 1000 images
Completed 2000 images
Completed 3000 images
Completed 4000 images
Completed 5000 images
Completed 6000 images
Completed 7000 images
Completed 8000 images


In [43]:
vgg11_features = images_data

In [44]:
vgg11_features['image features'] = vgg11_features.image.map(vgg11_image)

In [45]:
vgg11_features = vgg11_features[['image', 'image features']].reindex()
with open('Image features/vgg11.csv', mode='w', encoding='utf-8') as file: 
    vgg11_features.to_csv(file, index=False)

### **Load the VGG11BN**

In [46]:
# Load the pretrained model
vgg11_bn = models.vgg11_bn(pretrained=True)

vgg11_bn.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace=True)
    (14): MaxPool2d(ke

In [47]:
# Use the model obeject to select the desired layer
layer = vgg11_bn.classifier[-2]

In [48]:
resize = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])   # mean and std of ImageNet
to_tensor = transforms.ToTensor()

In [49]:
def get_vector_vgg11_bn(img_name):
    # Load the image with Pillow library 
    img = Image.open(img_name).convert('RGB')
    
    # Create a PyTorch Variable with the transformed image 
    transformed_img = Variable(normalize(to_tensor(resize(img))).unsqueeze(0))
    
    # Create a vector of zeros that will hold our feature vector 
    # The 'avgpool' layer has an output size of 4096
    my_embedding = torch.zeros(4096)
    
    # Define a function that will copy the output of a layer 
    def copy_data(m, i, o):
        my_embedding.copy_(o.data.view(-1))
        
    # Attach that function to our selected layer 
    h = layer.register_forward_hook(copy_data)
    
    # Run the model on our transformed image 
    vgg11_bn(transformed_img)
    
    # Detach our copy function from the layer 
    h.remove()
    
    # Return the feature vector 
    return my_embedding

In [50]:
vgg11_bn_image = {}
cnt = 0
for img in images_data.image:
    vgg11_bn_image[img] = get_vector_vgg11_bn(IMAGES_PATH + '/' + img)
    cnt += 1
    if cnt % 1000 == 0:
        print(f"Completed {cnt} images")

Completed 1000 images
Completed 2000 images
Completed 3000 images
Completed 4000 images
Completed 5000 images
Completed 6000 images
Completed 7000 images
Completed 8000 images


In [51]:
vgg11_bn_features = images_data

In [52]:
vgg11_bn_features['image features'] = vgg11_bn_features.image.map(vgg11_image)

In [53]:
vgg11_bn_features = vgg11_bn_features[['image', 'image features']].reindex()
with open('Image features/vgg11_bn.csv', mode='w', encoding='utf-8') as file: 
    vgg11_bn_features.to_csv(file, index=False)

### **Load the VGG13**

In [56]:
# load the pretrain model
vgg13 = models.vgg13(pretrained=True)

vgg13.eval()

Downloading: "https://download.pytorch.org/models/vgg13-19584684.pth" to C:\Users\ASUS/.cache\torch\hub\checkpoints\vgg13-19584684.pth
100%|██████████| 508M/508M [02:16<00:00, 3.90MB/s] 


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (15): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (16): 

In [57]:
# Use the model obeject to select the desired layer
layer = vgg13.classifier[-2]

In [58]:
resize = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])   # mean and std of ImageNet
to_tensor = transforms.ToTensor()

In [59]:
def get_vector_vgg13(img_name):
    # Load the image with Pillow library 
    img = Image.open(img_name).convert('RGB')
    
    # Create a PyTorch Variable with the transformed image 
    transformed_img = Variable(normalize(to_tensor(resize(img))).unsqueeze(0))
    
    # Create a vector of zeros that will hold our feature vector 
    # The 'avgpool' layer has an output size of 4096
    my_embedding = torch.zeros(4096)
    
    # Define a function that will copy the output of a layer 
    def copy_data(m, i, o):
        my_embedding.copy_(o.data.view(-1))
        
    # Attach that function to our selected layer 
    h = layer.register_forward_hook(copy_data)
    
    # Run the model on our transformed image 
    vgg13(transformed_img)
    
    # Detach our copy function from the layer 
    h.remove()
    
    # Return the feature vector 
    return my_embedding

In [60]:
vgg13_image = {}
cnt = 0
for img in images_data.image:
    vgg13_image[img] = get_vector_vgg13(IMAGES_PATH + '/' + img)
    cnt += 1
    if cnt % 1000 == 0:
        print(f"Completed {cnt} images")

Completed 1000 images
Completed 2000 images
Completed 3000 images
Completed 4000 images
Completed 5000 images
Completed 6000 images
Completed 7000 images
Completed 8000 images


In [61]:
vgg13_features = images_data

In [62]:
vgg13_features['image features'] = vgg13_features.image.map(vgg13_image)

In [63]:
vgg13_features = vgg13_features[['image', 'image features']].reindex()
with open('Image features/vgg13.csv', mode='w', encoding='utf-8') as file: 
    vgg13_features.to_csv(file, index=False)

### **Load the VGG13BN**

In [5]:
# load the pretrain model
vgg13_bn = models.vgg13_bn(pretrained=True)

vgg13_bn.eval()

Downloading: "https://download.pytorch.org/models/vgg13_bn-abd245e5.pth" to C:\Users\ASUS/.cache\torch\hub\checkpoints\vgg13_bn-abd245e5.pth
100%|██████████| 508M/508M [01:14<00:00, 7.13MB/s] 


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [6]:
# Use the model obeject to select the desired layer
layer = vgg13_bn.classifier[-2]

In [7]:
resize = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])   # mean and std of ImageNet
to_tensor = transforms.ToTensor()

In [8]:
def get_vector_vgg13_bn(img_name):
    # Load the image with Pillow library 
    img = Image.open(img_name).convert('RGB')
    
    # Create a PyTorch Variable with the transformed image 
    transformed_img = Variable(normalize(to_tensor(resize(img))).unsqueeze(0))
    
    # Create a vector of zeros that will hold our feature vector 
    # The 'avgpool' layer has an output size of 4096
    my_embedding = torch.zeros(4096)
    
    # Define a function that will copy the output of a layer 
    def copy_data(m, i, o):
        my_embedding.copy_(o.data.view(-1))
        
    # Attach that function to our selected layer 
    h = layer.register_forward_hook(copy_data)
    
    # Run the model on our transformed image 
    vgg13_bn(transformed_img)
    
    # Detach our copy function from the layer 
    h.remove()
    
    # Return the feature vector 
    return my_embedding

In [9]:
vgg13_bn_image = {}
cnt = 0
for img in images_data.image:
    vgg13_bn_image[img] = get_vector_vgg13_bn(IMAGES_PATH + '/' + img)
    cnt += 1
    if cnt % 1000 == 0:
        print(f"Completed {cnt} images")

Completed 1000 images
Completed 2000 images
Completed 3000 images
Completed 4000 images
Completed 5000 images
Completed 6000 images
Completed 7000 images
Completed 8000 images


In [10]:
vgg13_bn_features = images_data

In [11]:
vgg13_bn_features['image features'] = vgg13_bn_features.image.map(vgg13_bn_image)

In [12]:
vgg13_bn_features = vgg13_bn_features[['image', 'image features']].reindex()
with open('Image features/vgg13_bn.csv', mode='w', encoding='utf-8') as file: 
    vgg13_bn_features.to_csv(file, index=False)

### **Load the VGG16**

In [13]:
# load the pretrain model
vgg16 = models.vgg16(pretrained=True)

vgg16.eval()

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\ASUS/.cache\torch\hub\checkpoints\vgg16-397923af.pth
100%|██████████| 528M/528M [00:59<00:00, 9.29MB/s] 


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [14]:
# Use the model obeject to select the desired layer
layer = vgg16.classifier[-2]

In [15]:
resize = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])   # mean and std of ImageNet
to_tensor = transforms.ToTensor()

In [16]:
def get_vector_vgg16(img_name):
    # Load the image with Pillow library 
    img = Image.open(img_name).convert('RGB')
    
    # Create a PyTorch Variable with the transformed image 
    transformed_img = Variable(normalize(to_tensor(resize(img))).unsqueeze(0))
    
    # Create a vector of zeros that will hold our feature vector 
    # The 'avgpool' layer has an output size of 4096
    my_embedding = torch.zeros(4096)
    
    # Define a function that will copy the output of a layer 
    def copy_data(m, i, o):
        my_embedding.copy_(o.data.view(-1))
        
    # Attach that function to our selected layer 
    h = layer.register_forward_hook(copy_data)
    
    # Run the model on our transformed image 
    vgg16(transformed_img)
    
    # Detach our copy function from the layer 
    h.remove()
    
    # Return the feature vector 
    return my_embedding

In [17]:
vgg16_image = {}
cnt = 0
for img in images_data.image:
    vgg16_image[img] = get_vector_vgg16(IMAGES_PATH + '/' + img)
    cnt += 1
    if cnt % 1000 == 0:
        print(f"Completed {cnt} images")

Completed 1000 images
Completed 2000 images
Completed 3000 images
Completed 4000 images
Completed 5000 images
Completed 6000 images
Completed 7000 images
Completed 8000 images


In [18]:
vgg16_features = images_data

In [19]:
vgg16_features['image features'] = vgg16_features.image.map(vgg16_image)

In [20]:
vgg16_features = vgg16_features[['image', 'image features']].reindex()
with open('Image features/vgg16.csv', mode='w', encoding='utf-8') as file: 
    vgg16_features.to_csv(file, index=False)

### **Load the VGG16BN**

In [21]:
# load the pretrain model
vgg16_bn = models.vgg16_bn(pretrained=True)

vgg16_bn.eval()

Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to C:\Users\ASUS/.cache\torch\hub\checkpoints\vgg16_bn-6c64b313.pth
100%|██████████| 528M/528M [01:02<00:00, 8.84MB/s] 


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [22]:
# Use the model obeject to select the desired layer
layer = vgg16_bn.classifier[-2]

In [23]:
resize = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])   # mean and std of ImageNet
to_tensor = transforms.ToTensor()

In [24]:
def get_vector_vgg16_bn(img_name):
    # Load the image with Pillow library 
    img = Image.open(img_name).convert('RGB')
    
    # Create a PyTorch Variable with the transformed image 
    transformed_img = Variable(normalize(to_tensor(resize(img))).unsqueeze(0))
    
    # Create a vector of zeros that will hold our feature vector 
    # The 'avgpool' layer has an output size of 4096
    my_embedding = torch.zeros(4096)
    
    # Define a function that will copy the output of a layer 
    def copy_data(m, i, o):
        my_embedding.copy_(o.data.view(-1))
        
    # Attach that function to our selected layer 
    h = layer.register_forward_hook(copy_data)
    
    # Run the model on our transformed image 
    vgg16_bn(transformed_img)
    
    # Detach our copy function from the layer 
    h.remove()
    
    # Return the feature vector 
    return my_embedding

In [25]:
vgg16_bn_image = {}
cnt = 0
for img in images_data.image:
    vgg16_bn_image[img] = get_vector_vgg16_bn(IMAGES_PATH + '/' + img)
    cnt += 1
    if cnt % 1000 == 0:
        print(f"Completed {cnt} images")

Completed 1000 images
Completed 2000 images
Completed 3000 images
Completed 4000 images
Completed 5000 images
Completed 6000 images
Completed 7000 images
Completed 8000 images


In [26]:
vgg16_bn_features = images_data

In [27]:
vgg16_bn_features['image features'] = vgg16_bn_features.image.map(vgg16_bn_image)

In [28]:
vgg16_bn_features = vgg16_bn_features[['image', 'image features']].reindex()
with open('Image features/vgg16_bn.csv', mode='w', encoding='utf-8') as file: 
    vgg16_bn_features.to_csv(file, index=False)

### **Load the VGG19**

In [29]:
# load the pretrain model
vgg19 = models.vgg19(pretrained=True)

vgg19.eval()

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to C:\Users\ASUS/.cache\torch\hub\checkpoints\vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [01:07<00:00, 8.55MB/s] 


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [30]:
# Use the model obeject to select the desired layer
layer = vgg19.classifier[-2]

In [31]:
resize = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])   # mean and std of ImageNet
to_tensor = transforms.ToTensor()

In [32]:
def get_vector_vgg19(img_name):
    # Load the image with Pillow library 
    img = Image.open(img_name).convert('RGB')
    
    # Create a PyTorch Variable with the transformed image 
    transformed_img = Variable(normalize(to_tensor(resize(img))).unsqueeze(0))
    
    # Create a vector of zeros that will hold our feature vector 
    # The 'avgpool' layer has an output size of 4096
    my_embedding = torch.zeros(4096)
    
    # Define a function that will copy the output of a layer 
    def copy_data(m, i, o):
        my_embedding.copy_(o.data.view(-1))
        
    # Attach that function to our selected layer 
    h = layer.register_forward_hook(copy_data)
    
    # Run the model on our transformed image 
    vgg19(transformed_img)
    
    # Detach our copy function from the layer 
    h.remove()
    
    # Return the feature vector 
    return my_embedding

In [33]:
vgg19_image = {}
cnt = 0
for img in images_data.image:
    vgg19_image[img] = get_vector_vgg19(IMAGES_PATH + '/' + img)
    cnt += 1
    if cnt % 1000 == 0:
        print(f"Completed {cnt} images")

Completed 1000 images
Completed 2000 images
Completed 3000 images
Completed 4000 images
Completed 5000 images
Completed 6000 images
Completed 7000 images
Completed 8000 images


In [34]:
vgg19_features = images_data

In [35]:
vgg19_features['image features'] = vgg19_features.image.map(vgg19_image)

In [36]:
vgg19_features = vgg19_features[['image', 'image features']].reindex()
with open('Image features/vgg19.csv', mode='w', encoding='utf-8') as file: 
    vgg19_features.to_csv(file, index=False)

### **Load the VGG19BN**

In [37]:
# load the pretrain model
vgg19_bn = models.vgg19_bn(pretrained=True)

vgg19_bn.eval()

Downloading: "https://download.pytorch.org/models/vgg19_bn-c79401a0.pth" to C:\Users\ASUS/.cache\torch\hub\checkpoints\vgg19_bn-c79401a0.pth
100%|██████████| 548M/548M [01:37<00:00, 5.88MB/s] 


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [38]:
# Use the model obeject to select the desired layer
layer = vgg19_bn.classifier[-2]

In [39]:
resize = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])   # mean and std of ImageNet
to_tensor = transforms.ToTensor()

In [40]:
def get_vector_vgg19_bn(img_name):
    # Load the image with Pillow library 
    img = Image.open(img_name).convert('RGB')
    
    # Create a PyTorch Variable with the transformed image 
    transformed_img = Variable(normalize(to_tensor(resize(img))).unsqueeze(0))
    
    # Create a vector of zeros that will hold our feature vector 
    # The 'avgpool' layer has an output size of 4096
    my_embedding = torch.zeros(4096)
    
    # Define a function that will copy the output of a layer 
    def copy_data(m, i, o):
        my_embedding.copy_(o.data.view(-1))
        
    # Attach that function to our selected layer 
    h = layer.register_forward_hook(copy_data)
    
    # Run the model on our transformed image 
    vgg19_bn(transformed_img)
    
    # Detach our copy function from the layer 
    h.remove()
    
    # Return the feature vector 
    return my_embedding

In [41]:
vgg19_bn_image = {}
cnt = 0
for img in images_data.image:
    vgg19_bn_image[img] = get_vector_vgg19_bn(IMAGES_PATH + '/' + img)
    cnt += 1
    if cnt % 1000 == 0:
        print(f"Completed {cnt} images")

Completed 1000 images
Completed 2000 images
Completed 3000 images
Completed 4000 images
Completed 5000 images
Completed 6000 images
Completed 7000 images
Completed 8000 images


In [42]:
vgg19_bn_features = images_data

In [43]:
vgg19_bn_features['image features'] = vgg19_bn_features.image.map(vgg19_bn_image)

In [44]:
vgg19_bn_features = vgg19_bn_features[['image', 'image features']].reindex()
with open('Image features/vgg19_bn.csv', mode='w', encoding='utf-8') as file: 
    vgg19_bn_features.to_csv(file, index=False)